In [1]:
import pandas as pd
import numpy as np
import socket
import time

In [2]:
host = '95.217.155.82'
port = 8003

In [3]:
def make_request(host, port, request):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((host, port))

    byt = request.encode()
    s.send(byt)
    
    answer = request_loop(s)

    s.close()
    
    return answer

In [4]:
def request_loop(s, timeout=2):
    s.setblocking(0)
    total_data = []
    data = ''
    begin = time.time()

    while 1:
        if total_data and time.time()-begin > timeout:
            break
        elif time.time()-begin > timeout*2:
            break
        
        try:
            data = s.recv(4096)
            if data:
                data = data.decode('utf-8')
                total_data.append(data)
                begin = time.time()
            else:
                time.sleep(0.1)
        except:
            pass

    return ''.join(total_data)

In [5]:
def clean_ans(answer):
    answer = answer.splitlines()
    answer.pop(0)
    
    return answer

In [6]:
def create_list(answer):
    answerList = []
    if answer[0] == 'No results found.':
        return 'no'
    else:
        for i in answer:
            el = i.split('?')
            answerList.append(el)

    return answerList

In [7]:
methodDf = pd.read_csv('cleanedAuthorMethodV2.csv')
methodDf

,Unnamed: 0,Unnamed: 0.1,authorID,methodHash,projectID,projectVersion
0,0,0,e157cee2-b5b5-2466-14a8-717e9de1ce4c,b8a899ee5f7c93361e215edf5be05462,3477431694,1273494526000
1,1,1,a0c11df7-5118-e7d6-59c3-65223bf5b5a0,64132196448efd90223a4f7f16f0ae25,1821731652,1275264487000
2,2,2,a0c11df7-5118-e7d6-59c3-65223bf5b5a0,b8a76bcb30cd6afc1476ad39fc355317,949829554,1275264520000
3,9,9,a0c11df7-5118-e7d6-59c3-65223bf5b5a0,165944a511b4539a285f269aaac95437,2915245721,1407941931000
4,11,11,a0c11df7-5118-e7d6-59c3-65223bf5b5a0,165944a511b4539a285f269aaac95437,3881672,1278425588000
...,...,...,...,...,...,...
10426715,30250229,30250259,1ce151e9-89e4-c24e-19f4-a2528a2da121,37204df19ded9a9e3d8d982be9e9ea01,352558408,1455323653
10426716,30250231,30250261,1ce151e9-89e4-c24e-19f4-a2528a2da121,431f4e8f211d5f9a56fae1a0e8d47320,352558408,1426521636
10426717,30250233,30250263,1ce151e9-89e4-c24e-19f4-a2528a2da121,0ad0aaa9d2d046bfe7dc3aafc5f3566e,352558408,1444969846
10426718,30250234,30250264,edf67142-19da-8f47-ab69-6c1b37b94010,e606abc81be49f08071fc04e3386ad47,2011939588,1400533413


In [8]:
methodDf = methodDf.drop_duplicates(subset=['projectID', 'projectVersion'], keep='last')
methodDf = methodDf.sort_values('projectVersion')
methodDf = methodDf.drop_duplicates(subset=['projectID'], keep='last')
methodDf

,Unnamed: 0,Unnamed: 0.1,authorID,methodHash,projectID,projectVersion
9946626,28812713,28812741,778b73a2-8c93-c915-7ac8-cf5d8c031359,55931378a8b4199b49f9d33fc899322c,571223483,3599000
7429019,21216993,21217014,cb858566-e669-3f35-cedb-2e13adbdc5ff,67362e9ec33101ab83c3447afd9bc397,1566363144,839971503
8909672,25927550,25927573,246fe4fa-6859-642d-cc5a-ab076146159e,158186633cab1b2e2e4fd214d50dd217,2087511417,856536113
8909674,25927569,25927592,246fe4fa-6859-642d-cc5a-ab076146159e,158186633cab1b2e2e4fd214d50dd217,1480444098,856543260
9220202,26656443,26656466,9b9874d7-5c35-f65b-c296-0ceecd0e49dd,38a2e0158d450b9b6245aa593519beae,3217079470,931092840
...,...,...,...,...,...,...
9901958,28678243,28678271,6955ec40-5e37-b470-7279-f8ec1454cd9c,1761153af80a472b4b8c755b620590c8,1462075557,1646880547000
10359273,30061137,30061167,4243fd56-cf43-2d47-76c3-09e35b001ee2,696527c748152531745cad4c58551d2b,2341989400,1646939772000
10408083,30223020,30223050,e6e2cd2f-5113-7fec-32ca-5b373c0c5732,198cf98193260550224aa190785a84b8,658377018,1646962480000
2925038,8331331,8331343,3fd69006-22ef-9e94-9134-cc120a2b89ad,07271265e2a205d661a5a3b27eed9495,2309557017,1646971739000


In [9]:
methodListList = methodDf.values.tolist()
methodListList

[[28812713,
  28812741,
  '778b73a2-8c93-c915-7ac8-cf5d8c031359',
  '55931378a8b4199b49f9d33fc899322c',
  571223483,
  3599000],
 [21216993,
  21217014,
  'cb858566-e669-3f35-cedb-2e13adbdc5ff',
  '67362e9ec33101ab83c3447afd9bc397',
  1566363144,
  839971503],
 [25927550,
  25927573,
  '246fe4fa-6859-642d-cc5a-ab076146159e',
  '158186633cab1b2e2e4fd214d50dd217',
  2087511417,
  856536113],
 [25927569,
  25927592,
  '246fe4fa-6859-642d-cc5a-ab076146159e',
  '158186633cab1b2e2e4fd214d50dd217',
  1480444098,
  856543260],
 [26656443,
  26656466,
  '9b9874d7-5c35-f65b-c296-0ceecd0e49dd',
  '38a2e0158d450b9b6245aa593519beae',
  3217079470,
  931092840],
 [11622805,
  11622818,
  'd5d550c9-8a31-e223-690d-0124781e3d2e',
  '00fcdb8134ec0ca0821ec28a860a55fe',
  915414650,
  937488808],
 [4966303,
  4966310,
  '2764bbdb-532e-dd15-c1b8-79efcd6462b6',
  '16b79a4335e608b33a8c666006216fdf',
  2744092602,
  946529518],
 [26951957,
  26951982,
  'a27207c3-60f0-6d6b-8156-4e4763a913de',
  '242e05e8eb2c0

In [10]:
for methodList in methodListList:
    del methodList[:4]
methodListList

[[571223483, 3599000],
 [1566363144, 839971503],
 [2087511417, 856536113],
 [1480444098, 856543260],
 [3217079470, 931092840],
 [915414650, 937488808],
 [2744092602, 946529518],
 [43302738, 970481730],
 [727652210, 999170073],
 [30648175, 1000138793],
 [3152525216, 1009837232],
 [3500805271, 1049560485],
 [2136128910, 1050705917],
 [1967935258, 1053545842],
 [2603458841, 1064483930],
 [1027639780, 1064483930],
 [535537136, 1065753814],
 [1733401623, 1078188295],
 [1228548489, 1079190350],
 [1434230799, 1087220291],
 [1869210637, 1097152080],
 [959196368, 1102171019],
 [2447430193, 1106888803],
 [386458176, 1109548169],
 [3751762709, 1110411297],
 [1479641794, 1114421946],
 [1835717267, 1117477317],
 [1016340222, 1117477317],
 [820119296, 1117477317],
 [552297052, 1117477317],
 [1950082747, 1126790509],
 [1665311843, 1128379129],
 [1321156530, 1130009547],
 [3646430347, 1135636025],
 [2514973229, 1138223547],
 [124448766, 1138223547],
 [1082535660, 1138223547],
 [1089508334, 1139675739]

In [21]:
begin = time.time()
fullList = []
for i in range(0, len(methodListList), 100):
    callList = []
    listSlice = methodListList[i:i+100]
    for projectList in listSlice:
        projectList = list(map(str, projectList))
        project = '?'.join(projectList)
        callList.append(project)
    callString = '\n'.join(callList)
    callLength = str(len(callString))
    request = 'extp?seco_team2_21?' + callLength + '\n' + callString
    answer = make_request(host, port, request)
    if answer:
        cleanAns = clean_ans(answer)
        answerList = create_list(cleanAns)
        for projectInfo in answerList:
            fullList.append(projectInfo)
    else:
        print('skipped ', i, i+100)
    if (i + 100) % 1000 == 0:
        print('yes')
        if i > 0:
            dur = time.time() - begin
            timeToGo = ((len(methodListList) - i) * dur) / i
            hoursToGo = timeToGo / 60 / 60
            print(i, ' methods gedaan')
            print('Nog ongeveer', hoursToGo, ' uur te gaan')

projectDf = pd.DataFrame(fullList, columns=['projectID', 'version', 'versionHash', 'license', 'projectName', 'URL', 'ownerID', 'parserVersion'])

yes
900  methods gedaan
Nog ongeveer 0.24175838158042345  uur te gaan
yes
1900  methods gedaan
Nog ongeveer 0.21845614413141506  uur te gaan
yes
2900  methods gedaan
Nog ongeveer 0.20578550558049102  uur te gaan
yes
3900  methods gedaan
Nog ongeveer 0.19501113048781696  uur te gaan
yes
4900  methods gedaan
Nog ongeveer 0.18570057816721686  uur te gaan
yes
5900  methods gedaan
Nog ongeveer 0.1770663063352391  uur te gaan
yes
6900  methods gedaan
Nog ongeveer 0.168712675816291  uur te gaan
yes
7900  methods gedaan
Nog ongeveer 0.16059618410346546  uur te gaan
yes
8900  methods gedaan
Nog ongeveer 0.1531245550188679  uur te gaan
yes
9900  methods gedaan
Nog ongeveer 0.14541398784885504  uur te gaan
yes
10900  methods gedaan
Nog ongeveer 0.13779821313410603  uur te gaan
yes
11900  methods gedaan
Nog ongeveer 0.13116491096861221  uur te gaan
yes
12900  methods gedaan
Nog ongeveer 0.12437868952395893  uur te gaan
yes
13900  methods gedaan
Nog ongeveer 0.11682185810821519  uur te gaan
yes
149

In [22]:
projectDf.to_csv('projectInfo.csv')

In [23]:
projectDf

,projectID,version,versionHash,license,projectName,URL,ownerID,parserVersion
0,571223483,3599000,f7a64d744261d60c34a4bec1661c3f4d319161c1,-,-,-,4b56e262-856e-e2db-3813-da4603320d67,1
1,535537136,1065753814,26d833f1c8d0b2c0f4e0766e1567ec1961e79e3b,-,Lua-experimentation,https://github.com/ittner/Lua-experimentation,d981dba2-f196-3c74-57ac-c25640e168b6,1
2,1321156530,1130009547,51911976118447d663b5458453e91b9c36c7993e,-,jogl-demos,https://github.com/rsantina/jogl-demos,8c6fd8d5-3b63-567c-d009-718d48f87196,1
3,1064142182,1165888385,41b0bdedce3adeb921c181145d7c79a8ecbf4763,-,mootools-core,https://github.com/CrypticSwarm/mootools-core,37852619-b3de-91ac-8e13-832483f88d5c,1
4,3245398384,1186584194,fc8a99994d90d346f00fba0d0c5460694b4f81a4,"BSD 2-Clause ""Simplified"" License",sshkeychain,https://github.com/kogent/sshkeychain,5809fd30-10ba-f6d6-595e-72a0befd9282,1
...,...,...,...,...,...,...,...,...
29769,791395529,1646330540000,e2dcb868c445d9dcad41335c3aced2872b2f78ae,Apache License 2.0,mongo-python-driver,https://github.com/mongodb/mongo-python-driver,0464af90-e7f0-9f81-0ee1-3cdd786b0d2f,1
29770,351452968,1645188108000,0e9278a572379020bbed230bb0c78ce48a27e900,GNU General Public License v2.0,nodebox,https://github.com/nodebox/nodebox,f5ea34fd-141b-57f7-0901-f340206cb098,1
29771,552393469,1645845781000,2b1939d62a61113d2907ce8dee8f465e41940b2c,-,pyynikki,https://github.com/jarkko/pyynikki,3fde1180-456a-22c4-b5f5-a92fa813eadf,1
29772,2574829223,1646210537000,9f9992b56e0f1a69b6c323050cc9bba9ab797e0f,Other,libusual,https://github.com/libusual/libusual,a5153867-849e-c776-6713-ead4adb08e7c,1
